### install all dependincies and get ready enviroment

In [4]:
!pip install python-snappy

     |████████████████████████████████| 61kB 4.5MB/s eta 0:00:011


In [1]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [2]:
import tensorflow as tf

print('Installing TensorFlow Data Validation')
!pip install -q tensorflow_data_validation[visualization]

Installing TensorFlow Data Validation
     |████████████████████████████████| 788kB 7.8MB/s 
     |████████████████████████████████| 368kB 39.3MB/s 
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.


In [3]:
import pandas as pd
import tensorflow_data_validation as tfdev

### Simple data-analysis

In [7]:
dataset = pd.read_csv('/content/1.2 pollution_small.csv')

In [8]:
dataset.shape

(2188, 5)

In [9]:
training_data = dataset[:1600]
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [11]:
test_data = dataset[1600:]
test_data.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


### Data Analysis and validation using tfdev (tensorflow_data_validation)

#### Generate training data statistics

In [12]:
train_stats = tfdev.generate_statistics_from_dataframe(dataframe=dataset)

#### Infering the schema

In [13]:
schema = tfdev.infer_schema(statistics=train_stats)

In [14]:
tfdev.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


#### Calculate test set statistics

In [15]:
test_stats = tfdev.generate_statistics_from_dataframe(dataframe=test_data)

In [16]:
schema = tfdev.infer_schema(statistics=test_stats)

In [17]:
tfdev.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Compare test statistics with the schema 

#### checking for anomalies in new data

> any difference between test set and schema is called anomalies

In [19]:
anomalies = tfdev.validate_statistics(statistics=test_stats, schema=schema)

#### Display all detected anomalies

* integer larger than 10
* STRING type when expected INT type
* FLOAT type when expected INT type
* integer smaller than 0

In [20]:
tfdev.display_anomalies(anomalies)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


### New data WITH Anomalies

In [22]:
test_set_copy = test_data.copy()

In [23]:
test_set_copy.drop('soot', axis=1, inplace=True)

#### Statistics based on data with anomalies

In [24]:
test_set_copy_stats = tfdev.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [25]:
anomalies_new = tfdev.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [26]:
tfdev.display_anomalies(anomalies_new)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


### Prepare the schema for serving

In [30]:
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

#### Removing a target column from the serving schema

In [31]:
tfdev.get_feature(schema, 'soot').not_in_environment.append('SERVING')

#### Checking  for anomalies between the SERVING enviroment and new test set

In [32]:
serving_env_anomalies = tfdev.validate_statistics(test_set_copy_stats, schema, environment='SERVING')

In [33]:
tfdev.display_anomalies(serving_env_anomalies)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


### Freezing the schema

In [34]:
tfdev.write_schema_text(schema=schema, output_path='pollution_schema.pbtxt')